In [9]:

from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from opensearchpy.exceptions import TransportError, ConnectionError as OSCxnError

from requests_aws4auth import AWS4Auth
from openai import OpenAI, OpenAIError
from config import *
import time
import logging

from src.services.querying import llm_client, os_client, reconstruct_thread, answer_query, knn_search, Memory


In [10]:
# query = "How much do we ususally charge for traspots for 'compassionate funerals'?"

# hits, ids, q_vec = knn_search(query)

# for i , hit in enumerate(hits, start=1):
#     print(f"----Email thread {i}----")
#     print()
#     print(f"ID: {hit["_id"]}")
#     print(f"Score: {hit["_score"]}")
#     print(f"Subject: {hit["_source"]["subject"]}")
#     print(f"Summary: {hit["_source"]["summary_text"]}")
#     print()

In [11]:
retrieved_ids = []
memory = Memory(
    llm_client=llm_client,
    os_client=os_client,
    short_term_tokens=1000,
    mid_term_turns=3,
    memory_model=MEMORY_MODEL,
    embeddings_model=EMBEDDINGS_MODEL
)

In [19]:
bank=[
 "What how does Redcoat work with Lufthansa?",
 "Can you give specific examples?",
 "What are Lufthansa's rates to Africa?",
 "And what about other destinations, like US?",
 "Who is Simon and his role?",
 "How does he collaborate with Sarah Hawkins?",
 "Who else works there?",
 "If we were to compare US and African imports to the UK. How do they compare?",
 "Draft an SOP to import goods to the UK."
 ]

query = bank[-1]
#query = "How do you know this information?"
retrieved_ids = []  

prompt, response, mem_ctx, retrieved_ids, q_vec = answer_query(query, retrieved_ids=retrieved_ids, memory=memory)




----Rewritted query-----
 What are the key steps involved in importing goods to the UK according to Redcoat Express Ltd's practices sourced from email threads? 
----Rewritted query-----
 


In [20]:
import tiktoken

enc = tiktoken.encoding_for_model(QUERY_MODEL)

print(len(enc.encode(prompt)))

45463


In [21]:
#print(response)
print(response)

To draft a Standard Operating Procedure (SOP) for importing goods into the UK, we will outline the necessary steps and documentation required based on the practices and information available from Redcoat Express Ltd. This SOP will ensure compliance with UK customs regulations and facilitate smooth import operations.

### Standard Operating Procedure (SOP) for Importing Goods to the UK

#### 1. **Objective**
To establish a clear and efficient process for importing goods into the UK, ensuring compliance with customs regulations and minimizing delays.

#### 2. **Scope**
This SOP applies to all employees involved in the importation of goods to the UK through Redcoat Express Ltd.

#### 3. **Responsibilities**
- **Import Manager**: Oversee the import process and ensure compliance with regulations.
- **Logistics Coordinator**: Manage shipping logistics and documentation.
- **Customs Compliance Officer**: Ensure all customs documentation is accurate and submitted on time.

#### 4. **Procedure*

Conditional Retrieval

Cooldown TLL

Freshness Boosting

MRR (Penalizing documents which are too similar to what we already have)

Hierarchical Memory